In [19]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
from experiments.exp_long_term_forecasting_partial import Exp_Long_Term_Forecast_Partial
from experiments.exp_long_term_forecasting import Exp_Long_Term_Forecast
from data_provider.data_factory import data_provider
from model.iTransformer import Model
import random
import torch
import sys
import os
from utils.load import parse_bash_script, default_config, merge_args

fix_seed = 2024
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [20]:
bash_script_file = 'scripts/best_MS.sh'  # Replace with your bash script file name
with open(bash_script_file, 'r') as f:
    script_content = f.read()

bash_args = parse_bash_script(script_content)
bash_args.model_id

TypeError: Model.__init__() missing 1 required positional argument: 'configs'

In [ ]:
default_args = default_config().parse_args()
default_args

In [ ]:
args = merge_args(bash_args, default_args)
args

In [ ]:
setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des,
            args.class_strategy, 0)
setting

In [ ]:
exp = Exp_Long_Term_Forecast(args)
exp.model.load_state_dict(torch.load(os.path.join('./checkpoints/' + setting, 'checkpoint.pth')))

In [ ]:
exp.test(setting)

In [ ]:
args.is_training = 0 
exp = Exp_Long_Term_Forecast_Partial(args)
exp.model.load_state_dict(torch.load(os.path.join('./checkpoints/' + setting, 'checkpoint.pth')))
exp.predict(setting)

In [6]:
mae, mse, rmse, mape, mspe = np.load(os.path.join('results/' + setting, 'metrics.npy'))
preds = np.load(os.path.join('results/' + setting, 'pred.npy'))
trues = np.load(os.path.join('results/' + setting, 'true.npy'))
real_pred = np.load(os.path.join('results/' + setting, 'real_prediction.npy'))
train_loss = np.load(os.path.join('results/' + setting, 'train_loss.npy'))
val_loss = np.load(os.path.join('results/' + setting, 'val_loss.npy'))

In [8]:
real_pred

array([[[0.5891405 ],
        [0.5891405 ],
        [0.5891405 ],
        ...,
        [0.47864905],
        [0.47864905],
        [0.5368024 ]],

       [[0.5891405 ],
        [0.5891405 ],
        [0.5891405 ],
        ...,
        [0.47864905],
        [0.5368024 ],
        [0.5019104 ]],

       [[0.5891405 ],
        [0.5891405 ],
        [0.5891405 ],
        ...,
        [0.5368024 ],
        [0.5019104 ],
        [0.4902797 ]],

       ...,

       [[2.5430944 ],
        [2.54891   ],
        [2.54891   ],
        ...,
        [2.4791257 ],
        [2.3860803 ],
        [2.4093418 ]],

       [[2.54891   ],
        [2.54891   ],
        [2.5721712 ],
        ...,
        [2.3860803 ],
        [2.4093418 ],
        [2.4093418 ]],

       [[2.54891   ],
        [2.5721712 ],
        [2.4558644 ],
        ...,
        [2.4093418 ],
        [2.4093418 ],
        [2.4093418 ]]], dtype=float32)

In [9]:
%matplotlib inline
plt.plot(train_loss)
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

array([0.68322264, 0.60529286, 0.57409283, 0.56181302, 0.55694538])

In [10]:
plt.plot(val_loss)
plt.title('Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [11]:
import pandas as pd
df = pd.read_csv('D:/FPT/SU24/DSP391m/code/crawl/data/clean/df_combine.csv')
test_df = df.iloc[3914:]
y_test = test_df.Sell.to_list()
test_df

In [ ]:
# Flatten the 3D array to 1D and convert to list
pred_flattened = real_pred.flatten().tolist()
len(pred_flattened)

In [61]:
# Ensure y_test and pred_flattened are the same length
assert len(y_test) == len(pred_flattened), "Lengths of y_test and pred do not match!"

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='True Values', marker='o')
plt.plot(pred_flattened, label='Predicted Values', marker='x')
plt.title('True vs Predicted Values')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.show()